In [1]:
import os
import tensorflow as tf
import numpy as np

use_cpu = False

if use_cpu:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
    os.environ['CPU_ONLY'] = "TRUE"

    physical_devices = tf.config.list_physical_devices('CPU')

    tf.config.set_logical_device_configuration(
        physical_devices[0],
        [tf.config.LogicalDeviceConfiguration() for i in range(8)])
    logical_devices = tf.config.list_logical_devices('CPU')

    print(logical_devices)
else:
    os.environ['CPU_ONLY'] = "FALSE"
    physical_devices = tf.config.list_physical_devices('GPU')
    print(physical_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
from latentneural.data import DataManager
from latentneural import LFADS

data_dir = os.path.join('..', 'latentneural', 'data', 'storage', 'lorenz', '20210610T215300')
dataset, settings = DataManager.load_dataset(
    directory=data_dir,
    filename='dataset.h5')

In [3]:
neural_data = dataset['train_data'].astype('float')

In [4]:
targets = tf.cast(neural_data, dtype=tf.float32)
log_f = tf.cast(np.log(settings['step'] * dataset['train_rates']), tf.float32)

loss_tf_r = tf.nn.log_poisson_loss(
    targets=targets, 
    log_input=log_f, compute_full_loss=True
).numpy()

loss_math_r = - (targets * log_f - tf.exp(log_f) - tf.math.lgamma(targets + 1))


loss_tf = np.sum(loss_tf_r)
loss_math = np.sum(loss_math_r)
print(loss_tf)
print(loss_math)

48137.016
48137.11


In [5]:
log_f = np.array([-1e+20], dtype=np.float32)
targets = np.array([1e+10], dtype=np.float32)

print(tf.nn.log_poisson_loss(log_input=log_f, targets=targets).numpy())
print((- (targets * log_f - tf.exp(log_f) - tf.math.lgamma(targets + 1))).numpy())

[1.e+30]
[1.e+30]


In [6]:
log_input=np.array([1e+38], dtype=np.float32)
targets=np.array([1e+38], dtype=np.float32)

logp = targets * log_input - tf.exp(log_input) - tf.math.lgamma(targets + 1)

logp

<ipython-input-6-d73ab3d65deb>:4: RuntimeWarning: overflow encountered in multiply
  logp = targets * log_input - tf.exp(log_input) - tf.math.lgamma(targets + 1)


<tf.Tensor: shape=(1,), dtype=float32, numpy=array([nan], dtype=float32)>

In [7]:
tf.math.lgamma(targets + 1)

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([inf], dtype=float32)>

In [8]:
tf.exp(log_input)

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([inf], dtype=float32)>